In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
sys.path.append('/content/drive/MyDrive/cv_project_fall_2022')

In [3]:
import os
import torch
from torch.utils.data import DataLoader

import utils.utils as utils
from models.definitions.transformer_net_new import TransformerNet

import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms

In [4]:
def stylize_static_image(inference_config):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Prepare the model - load the weights and put the model into evaluation mode
    stylization_model = TransformerNet().to(device)
    model_saved_path = inference_config['model_saved_path']
    training_state = torch.load(model_saved_path)
    state_dict = training_state["state_dict"]
    stylization_model.load_state_dict(state_dict, strict=True)
    stylization_model.eval()

    if inference_config['verbose']:
        utils.print_model_metadata(training_state)

    content_img_path = inference_config['content_images_path']
    print(content_img_path)
    content_image_to_show = Image.open(content_img_path)
    plt.imshow(content_image_to_show)
    plt.show()

    

    with torch.no_grad():
        transform_list = [transforms.ToTensor()]
        transform = transforms.Compose(transform_list)
        
        target_transform_list = [transforms.ToTensor()]
        #target_transform_list = [transforms.ToTensor(), transforms.GaussianBlur(kernel_size=(51, 51), sigma=(70, 70))]
        transform_target = transforms.Compose(target_transform_list)
        
        img = utils.load_image(content_img_path, target_shape=inference_config['img_width'])
        img = transform(img).to(device)

        face_masked_image_batch = utils.load_image(inference_config['content_images_mask_path'], target_shape=inference_config['img_width'], RGB=False)
        face_masked_image_batch = transform_target(face_masked_image_batch).to(device)
        face_masked_image_batch = face_masked_image_batch * 0.03

        result = torch.cat([img, face_masked_image_batch], dim=0)

        print(' Infering...')
        print(img.shape)
        content_image = result.repeat(1, 1, 1, 1)
        print(content_image.shape)
        print(' Infering Ends...')
        stylized_img = stylization_model(content_image).to('cpu').numpy()[0]
        utils.save_and_maybe_display_image(inference_config, stylized_img, should_display=inference_config['display'])


In [12]:
inference_config = dict()

# Content image(s) to stylize
inference_config['content_input'] = 'lion.jpg'
# Resize content image to this width
inference_config['img_width'] = 500
# Set to True to print Model Information
inference_config['verbose'] = True
# Display Result 
inference_config['display'] = True
# Output Images Path 
inference_config['redirected_output'] = os.path.join('/content/drive/MyDrive/cv_project_fall_2022', 'data', 'output-images')


# 1st conv layer
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_content_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth'
# 3rd conv layer
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_content_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'
# 1st conv layer
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_1st_layer_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'

# 2nd conv layer
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_1st_layer_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'

# 2nd conv layer - using content and style loss of layer 2
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/ckpt_style_face_layer_2_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'

inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_depth_face_edge_layer_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_0_batch_1399.pth'

# Depth and Face Edge - Final
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_depth_face_edge_layer_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'

# Face Edge Only Mosaic
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_face_edge_only_new_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_6_batch_1899.pth'

# Face Edge Only - Final
#inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_face_edge_only_ldc_pre_mask_new_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth'

# Test 
#inference_config['model_saved_path'] = model_binaries_path
inference_config['model_saved_path'] = "/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/blurred_ckpt_style_face_layer_1_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_0_batch_1699.pth"
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/blurred_ckpt_style_face_layer_1_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_499.pth'
#inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_face_layer_2_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'

#inference_config['model_saved_path'] = "/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_3rd_layer_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth"


#inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'



'''Mosaic'''
# Style Image Path
inference_config['style_images_path'] = '/content/drive/MyDrive/cv_project_fall_2022/data/style-images/mosaic.jpg'
# [DONE] Style Transfer on Face using blurred Style Image + Edge Loss
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/blurred_ckpt_style_face_layer_1_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_3_batch_1899.pth'
# Saving checkpoint :  /content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/blurred_ckpt_style_face_layer_1_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth
# [TODO] Edge Loss
#inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/mosaic/ckpt_style_face_edge_only_new_mosaic_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth'
# [TODO]  Content Preserve + Content Style Transfer on Face

'''Edtaonist'''
inference_config['style_images_path'] = '/content/drive/MyDrive/cv_project_fall_2022/data/style-images/edtaonisl.jpg'
# [DONE] Style Transfer on Face using blurred Style Image + Edge Loss
inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl-blurred-style-image-mask_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth'

# [DONE] Style Transfer on Face using blurred Style Image + Edge Loss
#inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl-blurred-style-image-mask_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth'
# [TODO] Edge Loss
# [TODO] Content Preserve + Content Style Transfer on Face
#inference_config['model_saved_path'] = '/content/drive/MyDrive/cv_project_fall_2022/models/checkpoints/edtaonisl/ckpt_style_face_content_edtaonisl_cw_1.0_sw_400000.0_tw_0_epoch_9_batch_1899.pth'



os.makedirs(inference_config['redirected_output'], exist_ok=True)
test_images_path = '/content/drive/MyDrive/cv_project_fall_2022/face_mask/test_images/'
all_test_images = os.listdir(test_images_path)


def get_mask_path(file_path):
    file_name = file_path.split('/')[-1]
    file_name_split = file_name.split('_')
    if(file_name_split[1] == 'Election'):
        file_name_split[1] = 'Election_Campain'
    if(file_name_split[1] == 'Press'):
        file_name_split[1] = 'Press_Conference'
    
    

    dir_name = file_name_split[0] + '--' + file_name_split[1]
    return file_path.replace("/test_images/", "/data/widerface/WIDER_train/masks/" + dir_name + "/")

# Show style Image
style_image_path = inference_config['style_images_path']
style_image_to_show = Image.open(style_image_path)
plt.imshow(style_image_to_show)
plt.show()

for test_image in all_test_images:
    inference_config['content_images_path'] = test_images_path + test_image
    inference_config['content_images_mask_path'] = get_mask_path(inference_config['content_images_path'])
    stylize_static_image(inference_config)



Output hidden; open in https://colab.research.google.com to view.